In [5]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Bazowy URL API BDL
BASE_URL = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{var_id}"

# Jeśli masz klucz API (X-ClientId), możesz go wpisać tutaj.
# Jak nie masz – zostaw None, większość prostych zapytań i tak przechodzi.
API_KEY = None  # np. "twoj_klucz_api"

# Mapowanie poziomów jednostek:
# 0 – Polska, 2 – województwa, 5 – powiaty :contentReference[oaicite:1]{index=1}
LEVEL_POLSKA = 0
LEVEL_WOJ = 2
LEVEL_POW = 5


In [6]:
def fetch_bdl_xml(var_id, unit_level, years, page_size=100):
    """
    Pobiera dane z BDL dla zadanej zmiennej, poziomu jednostek i listy lat.
    Zwraca pandas.DataFrame z kolumnami:
    unit_level, unit_id, unit_name, year, value
    """
    all_records = []

    headers = {}
    if API_KEY is not None:
        headers["X-ClientId"] = API_KEY

    for year in years:
        page = 0
        while True:
            params = {
                "format": "xml",
                "year": year,
                "unit-level": unit_level,
                "page-size": page_size,
                "page": page,
            }

            url = BASE_URL.format(var_id=var_id)
            response = requests.get(url, params=params, headers=headers)
            response.raise_for_status()

            xml_data = response.text
            root = ET.fromstring(xml_data)

            # Wszystkie jednostki na tej stronie
            units = root.findall(".//unitData")
            if not units:
                # Brak danych = koniec stron dla tego roku
                break

            for unit in units:
                unit_id_el = unit.find("id")
                unit_name_el = unit.find("name")

                unit_id = unit_id_el.text if unit_id_el is not None else None
                unit_name = unit_name_el.text if unit_name_el is not None else None

                # W środku unitData są yearVal dla każdego roku (tu zwykle 1, bo filtrujemy po year)
                for year_val in unit.findall(".//yearVal"):
                    year_el = year_val.find("year")
                    val_el = year_val.find("val")

                    year_value = int(year_el.text) if year_el is not None else year

                    val_text = val_el.text if val_el is not None else None
                    if val_text is None:
                        value = None
                    else:
                        # Na wszelki wypadek, gdyby przecinek był separatorem dziesiętnym
                        try:
                            value = float(val_text.replace(",", "."))
                        except ValueError:
                            value = None

                    all_records.append({
                        "unit_level": unit_level,
                        "unit_id": unit_id,
                        "unit_name": unit_name,
                        "year": year_value,
                        "value": value,
                    })

            page += 1

    df = pd.DataFrame(all_records)
    return df


In [7]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# -----------------------
# KONFIGURACJA BDL API
# -----------------------

# Endpoint "data/by-variable" z dokumentacji API BDL
BASE_URL = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{var_id}"

# ID zmiennej: Mediana cen za 1 m2 lokali mieszkalnych sprzedanych w ramach transakcji rynkowych (P3787)
# z wymiarami: Transakcje rynkowe (rynek pierwotny), Powierzchnia użytkowa (ogółem)
VAR_ID = "3787"

# Klucz API – NAJPEWNIEJ zostawisz None, jeśli nie masz swojego X-ClientId.
# Dokumentacja: nagłówek X-ClientId w requestach do BDL API. :contentReference[oaicite:1]{index=1}
API_KEY = None  # np. "TWÓJ_KLUCZ_API" – jak masz

# Poziomy jednostek terytorialnych (unit-level):
# 0 – Polska, 2 – województwa, 5 – powiaty (patrz dokumentacja unitLevel/unit-level) :contentReference[oaicite:2]{index=2}
LEVEL_POLSKA = 0
LEVEL_WOJ = 2
LEVEL_POW = 5

# Zakres lat do pobrania
YEARS = list(range(2014, 2025))  # 2014–2024


In [8]:
def fetch_bdl_xml_for_variable(
    var_id: str,
    unit_level: int,
    years,
    page_size: int = 100,
):
    """
    Pobiera dane z BDL API (data/by-variable) dla:
    - konkretnej zmiennej (var_id),
    - poziomu jednostek (unit_level),
    - listy lat (years).

    Zwraca DataFrame z kolumnami:
    unit_level, unit_id, unit_name, year, value
    """
    all_records = []

    headers = {}
    if API_KEY is not None:
        headers["X-ClientId"] = API_KEY

    for year in years:
        page = 0

        while True:
            params = {
                "format": "xml",
                "year": year,
                "unit-level": unit_level,  # nazwa parametru z dokumentacji API BDL
                "page": page,
                "page-size": page_size,
            }

            url = BASE_URL.format(var_id=var_id)
            resp = requests.get(url, params=params, headers=headers)
            resp.raise_for_status()

            xml_data = resp.text
            root = ET.fromstring(xml_data)

            units = root.findall(".//unitData")
            if not units:
                # brak danych = koniec stron dla danego roku
                break

            for unit in units:
                unit_id_el = unit.find("id")
                unit_name_el = unit.find("name")

                unit_id = unit_id_el.text if unit_id_el is not None else None
                unit_name = unit_name_el.text if unit_name_el is not None else None

                # yearVal – zwykle 1 na rekord, bo filtrujemy po year,
                # ale struktura XML dopuszcza więcej.
                for year_val in unit.findall(".//yearVal"):
                    year_el = year_val.find("year")
                    val_el = year_val.find("val")

                    year_value = int(year_el.text) if year_el is not None else year

                    if val_el is None or val_el.text is None:
                        value = None
                    else:
                        # zamiana ewentualnego przecinka dziesiętnego na kropkę
                        txt = val_el.text.strip()
                        try:
                            value = float(txt.replace(",", "."))
                        except ValueError:
                            value = None

                    all_records.append(
                        {
                            "unit_level": unit_level,
                            "unit_id": unit_id,
                            "unit_name": unit_name,
                            "year": year_value,
                            "value": value,
                        }
                    )

            page += 1

    df = pd.DataFrame(all_records)
    return df


In [9]:
from pathlib import Path

# Katalogi na dane
DATA_RAW = Path("data/raw")
DATA_PROCESSED = Path("data/processed")
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

# --- POLSKA (poziom 0) ---
polska_df = fetch_bdl_xml_for_variable(
    var_id=VAR_ID,
    unit_level=LEVEL_POLSKA,
    years=YEARS,
    page_size=100,
)

# --- WOJEWÓDZTWA (poziom 2) ---
woj_df = fetch_bdl_xml_for_variable(
    var_id=VAR_ID,
    unit_level=LEVEL_WOJ,
    years=YEARS,
    page_size=100,
)

# --- POWIATY (poziom 5) ---
powiaty_df = fetch_bdl_xml_for_variable(
    var_id=VAR_ID,
    unit_level=LEVEL_POW,
    years=YEARS,
    page_size=100,
)

print("Polska:")
display(polska_df.head())

print("Województwa:")
display(woj_df.head())

print("Powiaty:")
display(powiaty_df.head())


Polska:


,unit_level,unit_id,unit_name,year,value
0,0,000000000000,POLSKA,2014,35.0
1,0,000000000000,POLSKA,2016,38.0
2,0,000000000000,POLSKA,2017,34.0
3,0,000000000000,POLSKA,2018,40.0
4,0,000000000000,POLSKA,2019,42.0


Województwa:


,unit_level,unit_id,unit_name,year,value
0,2,011200000000,MAŁOPOLSKIE,2014,3.0
1,2,012400000000,ŚLĄSKIE,2014,8.0
2,2,020800000000,LUBUSKIE,2014,1.0
3,2,023000000000,WIELKOPOLSKIE,2014,0.0
4,2,023200000000,ZACHODNIOPOMORSKIE,2014,1.0


Powiaty:


""


In [12]:
def tidy_country(df):
    import pandas as pd
    if df is None or getattr(df, "empty", True):
        return pd.DataFrame(columns=["year", "level", "unit_id", "unit_name", "price_median"])
    df = df.copy()
    df["level"] = "Polska"
    df.rename(columns={"value": "price_median"}, inplace=True)
    cols = ["year", "level", "unit_id", "unit_name", "price_median"]
    for c in cols:
        if c not in df.columns:
            df[c] = pd.NA
    return df[cols]


def tidy_woj(df):
    import pandas as pd
    if df is None or getattr(df, "empty", True):
        return pd.DataFrame(columns=["year", "level", "unit_code", "unit_name", "price_median"])
    df = df.copy()
    df["level"] = "województwo"
    df.rename(columns={"value": "price_median", "unit_id": "unit_code"}, inplace=True)
    cols = ["year", "level", "unit_code", "unit_name", "price_median"]
    for c in cols:
        if c not in df.columns:
            df[c] = pd.NA
    return df[cols]


def tidy_powiaty(df):
    import pandas as pd
    if df is None or getattr(df, "empty", True):
        return pd.DataFrame(columns=["year", "level", "unit_code", "unit_name", "price_median"])
    df = df.copy()
    df["level"] = "powiat"
    df.rename(columns={"value": "price_median", "unit_id": "unit_code"}, inplace=True)
    cols = ["year", "level", "unit_code", "unit_name", "price_median"]
    for c in cols:
        if c not in df.columns:
            df[c] = pd.NA
    return df[cols]


polska_clean = tidy_country(polska_df)
woj_clean = tidy_woj(woj_df)
powiaty_clean = tidy_powiaty(powiaty_df)

print("Polska (po clean):")
display(polska_clean.head())

print("Województwa (po clean):")
display(woj_clean.head())

print("Powiaty (po clean):")
display(powiaty_clean.head())

# Zapis do plików, których będziesz używać w kolejnych notebookach
from pathlib import Path
DATA_PROCESSED = Path("data/processed")
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
polska_clean.to_csv(DATA_PROCESSED / "polska_median_primary_2014_2024.csv", index=False)
woj_clean.to_csv(DATA_PROCESSED / "woj_median_primary_2014_2024.csv", index=False)
powiaty_clean.to_csv(DATA_PROCESSED / "powiaty_median_primary_2014_2024.csv", index=False)


Polska (po clean):


,year,level,unit_id,unit_name,price_median
0,2014,Polska,000000000000,POLSKA,35.0
1,2016,Polska,000000000000,POLSKA,38.0
2,2017,Polska,000000000000,POLSKA,34.0
3,2018,Polska,000000000000,POLSKA,40.0
4,2019,Polska,000000000000,POLSKA,42.0


Województwa (po clean):


,year,level,unit_code,unit_name,price_median
0,2014,województwo,011200000000,MAŁOPOLSKIE,3.0
1,2014,województwo,012400000000,ŚLĄSKIE,8.0
2,2014,województwo,020800000000,LUBUSKIE,1.0
3,2014,województwo,023000000000,WIELKOPOLSKIE,0.0
4,2014,województwo,023200000000,ZACHODNIOPOMORSKIE,1.0


Powiaty (po clean):


,year,level,unit_code,unit_name,price_median


In [11]:
# Diagnoza struktury kolumn
print("polska_df columns:", list(polska_df.columns))
print(polska_df.head(2))
print("\nwoj_df columns:", list(woj_df.columns))
print(woj_df.head(2))
print("\npowiaty_df columns:", list(powiaty_df.columns))
print(powiaty_df.head(2))

polska_df columns: ['unit_level', 'unit_id', 'unit_name', 'year', 'value']
   unit_level       unit_id unit_name  year  value
0           0  000000000000    POLSKA  2014   35.0
1           0  000000000000    POLSKA  2016   38.0

woj_df columns: ['unit_level', 'unit_id', 'unit_name', 'year', 'value']
   unit_level       unit_id    unit_name  year  value
0           2  011200000000  MAŁOPOLSKIE  2014    3.0
1           2  012400000000      ŚLĄSKIE  2014    8.0

powiaty_df columns: []
Empty DataFrame
Columns: []
Index: []


In [13]:
# Podgląd danych – surowe i po czyszczeniu
from IPython.display import display

try:
    print("Surowe dane – Polska:", polska_df.shape)
    display(polska_df.head(20))
except Exception as e:
    print("Brak polska_df:", e)

try:
    print("Surowe dane – Województwa:", woj_df.shape)
    display(woj_df.head(20))
except Exception as e:
    print("Brak woj_df:", e)

try:
    print("Surowe dane – Powiaty:", powiaty_df.shape)
    display(powiaty_df.head(20))
except Exception as e:
    print("Brak powiaty_df:", e)

print("\n=== Po clean/tidy ===")
try:
    print("Polska (clean):", polska_clean.shape)
    display(polska_clean.sort_values(["year"]).head(20))
except Exception as e:
    print("Brak polska_clean:", e)

try:
    print("Województwa (clean):", woj_clean.shape)
    display(woj_clean.sort_values(["year","unit_name"]).head(40))
except Exception as e:
    print("Brak woj_clean:", e)

try:
    print("Powiaty (clean):", powiaty_clean.shape)
    display(powiaty_clean.head(20))
except Exception as e:
    print("Brak powiaty_clean:", e)


Surowe dane – Polska: (10, 5)


,unit_level,unit_id,unit_name,year,value
0,0,000000000000,POLSKA,2014,35.0
1,0,000000000000,POLSKA,2016,38.0
2,0,000000000000,POLSKA,2017,34.0
3,0,000000000000,POLSKA,2018,40.0
4,0,000000000000,POLSKA,2019,42.0
5,0,000000000000,POLSKA,2020,22.0
6,0,000000000000,POLSKA,2021,22.0
7,0,000000000000,POLSKA,2022,32.0
8,0,000000000000,POLSKA,2023,32.0
9,0,000000000000,POLSKA,2024,38.0


Surowe dane – Województwa: (160, 5)


,unit_level,unit_id,unit_name,year,value
0,2,011200000000,MAŁOPOLSKIE,2014,3.0
1,2,012400000000,ŚLĄSKIE,2014,8.0
2,2,020800000000,LUBUSKIE,2014,1.0
3,2,023000000000,WIELKOPOLSKIE,2014,0.0
4,2,023200000000,ZACHODNIOPOMORSKIE,2014,1.0
5,2,030200000000,DOLNOŚLĄSKIE,2014,6.0
6,2,031600000000,OPOLSKIE,2014,0.0
7,2,040400000000,KUJAWSKO-POMORSKIE,2014,3.0
8,2,042200000000,POMORSKIE,2014,1.0
9,2,042800000000,WARMIŃSKO-MAZURSKIE,2014,2.0


Surowe dane – Powiaty: (0, 0)


...

,year,level,unit_id,unit_name,price_median
0,2014,Polska,000000000000,POLSKA,35.0
1,2016,Polska,000000000000,POLSKA,38.0
2,2017,Polska,000000000000,POLSKA,34.0
3,2018,Polska,000000000000,POLSKA,40.0
4,2019,Polska,000000000000,POLSKA,42.0
5,2020,Polska,000000000000,POLSKA,22.0
6,2021,Polska,000000000000,POLSKA,22.0
7,2022,Polska,000000000000,POLSKA,32.0
8,2023,Polska,000000000000,POLSKA,32.0
9,2024,Polska,000000000000,POLSKA,38.0


Województwa (clean): (160, 5)


,year,level,unit_code,unit_name,price_median
5,2014,województwo,030200000000,DOLNOŚLĄSKIE,6.0
7,2014,województwo,040400000000,KUJAWSKO-POMORSKIE,3.0
12,2014,województwo,060600000000,LUBELSKIE,0.0
2,2014,województwo,020800000000,LUBUSKIE,1.0
15,2014,województwo,071400000000,MAZOWIECKIE,7.0
0,2014,województwo,011200000000,MAŁOPOLSKIE,3.0
6,2014,województwo,031600000000,OPOLSKIE,0.0
13,2014,województwo,061800000000,PODKARPACKIE,0.0
14,2014,województwo,062000000000,PODLASKIE,0.0
8,2014,województwo,042200000000,POMORSKIE,1.0


Powiaty (clean): (0, 5)


,year,level,unit_code,unit_name,price_median


In [ ]:
# Diagnostyka: sprawdzenie dostępności danych dla poziomów 0/2/5
import requests

def probe_levels(var_id: str, sample_years=(2022,), levels=(0,2,5), page_size=1, lang="pl"):
    base = "https://bdl.stat.gov.pl/api/v1/data/by-variable/{var_id}"
    for lvl in levels:
        for y in sample_years:
            url = base.format(var_id=var_id)
            params = {
                "format": "json",
                "year": [y],
                "unit-level": lvl,
                "page-size": page_size,
                "page": 0,
                "lang": lang,
            }
            try:
                r = requests.get(url, params=params, timeout=20)
                r.raise_for_status()
                data = r.json()
                units = data.get("results") or data.get("data") or []
                print(f"level={lvl}, year={y} -> rekordów: {len(units)}")
            except Exception as e:
                print(f"level={lvl}, year={y} -> błąd: {e}")

print("Sprawdzam poziomy dla VAR_ID=", VAR_ID)
probe_levels(VAR_ID, sample_years=(2022, 2024), levels=(0,2,5))

Sprawdzam poziomy dla VAR_ID= 3787
level=0, year=2022 -> rekordów: 1
level=0, year=2024 -> rekordów: 1
level=2, year=2022 -> rekordów: 1
level=2, year=2024 -> rekordów: 1
